In [0]:
%run ./00_setup_environment.ipynb

In [0]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.chains import RetrievalQA
from langchain_openai.chat_models import ChatOpenAI


In [0]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

#  OpenAI LLM (requires API key)
from langchain_openai import ChatOpenAI

# Optional: dotenv to load keys safely
import os
from dotenv import load_dotenv


In [0]:
index_path = "/Volumes/genai_catalog/car_manuals/manual_chunks/faiss_index"

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/distiluse-base-multilingual-cased-v2"
)

vectorstore = FAISS.load_local(
    index_path,
    embedding_model,
    allow_dangerous_deserialization=True #You should only enable this if you're loading a FAISS index that you created and trust — which is the case here.
)

# ---- Initialize LLM ----
# OpenAI model (paid)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")  # load from env
)



In [0]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

multilingual_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a helpful assistant that answers questions about Audi car manuals. Use only the information provided in the context below to answer the question.

Please answer in the **same language** as the question. If you do not know the answer based on the context, just say you don't know.

--- CONTEXT START ---
{context}
--- CONTEXT END ---

Question: {question}

Answer:
"""
)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": multilingual_prompt}
)


In [0]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# ---- Sample Query ----
query = "What is Audi pre sense front and how does it work?"
# query = "How do I activate the Audi adaptive cruise control?"
# query = "Wie funktioniert Audi pre sense front?"
# query = 'Wie aktiviere ich den adaptiven Tempomaten (Adaptive Cruise Control)?'


result = rag_chain.invoke(query)

print("Answer:", result['result'])

# (Optional) print sources
for doc in result['source_documents']:
    print("\n📄 Source:", doc.metadata.get("source", "unknown"))
